## SetFit model training using the created JSON file 
#### 'breaking_bad_analysisV2.json' from (M2_LLM_Data_Fetch_and_Processing.ipynb)

 * Model used: sentence-transformers/paraphrase-mpnet-base-v2

 * Train/Test split: 80/20

 * Runtime (Colab GPU T4): Aprox. 8 minuttes 

 * Make sure to use GPU for training and testing

 * Saves trained model in the folder 'saved_model'

### Install & Import Libraries needed for model training 🎛️

In [3]:
# Install required packages from requirements.txt
!pip install -r https://raw.githubusercontent.com/Markushenriksson13/NLP-and-Network-Analysis_Exam_Submission/refs/heads/main/requirements.txt -q

# import libs
import json
import pandas as pd
from datasets import Dataset
from setfit import SetFitModel, SetFitTrainer
from sklearn.metrics import classification_report
import requests

/opt/anaconda3/lib/python3.12/pty.py:95: DeprecationWarning: This process (pid=90396) is multi-threaded, use of forkpty() may lead to deadlocks in the child.
  pid, fd = os.forkpty()


### SetFit model for training 🧮 

In [4]:
# ONLINE DOWNLOADING JSON file from Github repository
# URL to JSON-file
url = 'https://raw.githubusercontent.com/Markushenriksson13/NLP-and-Network-Analysis_Exam_Submission/main/breaking_bad_analysisV2.json'

# Download JSON-file
response = requests.get(url)
data = response.json()  # Convert to JSON-format

In [5]:
# OFFLINE LOADING OF JSON:
#with open("breaking_bad_analysisV2.json", 'r', encoding='utf-8') as file:
#    data = json.load(file)

In [6]:
def load_and_prepare_data():
    """Load JSON and prepare data for classification"""
    
    relationships = []
    labels = []

    for episode in data.values():
        for rel in episode.get('relationships', []):
            relationships.append(f"{rel['source']} - {rel['target']}")
            labels.append(rel['relation'])

    # Split into train/test (80/20)
    df = pd.DataFrame({'text': relationships, 'label': labels})
    train_size = int(len(df) * 0.8)

    train_data = Dataset.from_pandas(df[:train_size])
    test_data = Dataset.from_pandas(df[train_size:])

    return train_data, test_data

def train_and_evaluate():
    # load and prepare data
    train_dataset, test_dataset = load_and_prepare_data()

    print(f"Training samples: {len(train_dataset)}")
    print(f"Testing samples: {len(test_dataset)}")

    # start and train model
    model = SetFitModel.from_pretrained("sentence-transformers/paraphrase-mpnet-base-v2")
    trainer = SetFitTrainer(
        model=model,
        train_dataset=train_dataset,
        batch_size=16,
        num_iterations=20,
        num_epochs=1
    )

    trainer.train()

    # evaluate
    predictions = model.predict(test_dataset['text'])
    print("\nClassification Report:")
    print(classification_report(test_dataset['label'], predictions))

    # Example predictions
    print("\nExample Predictions:")
    for text, true_label, pred_label in zip(
        test_dataset['text'][:3],
        test_dataset['label'][:3],
        predictions[:3]
    ):
        print(f"\nText: {text}")
        print(f"True: {true_label}")
        print(f"Predicted: {pred_label}")

    # save Model
    model.save_pretrained("saved_model")
    
    return model

# run train & evaluation
model = train_and_evaluate()

Training samples: 475
Testing samples: 119


/opt/anaconda3/lib/python3.12/site-packages/transformers/tokenization_utils_base.py:1617: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be deprecated in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(
model_head.pkl not found on HuggingFace Hub, initialising classification head with random weights. You should TRAIN this model on a downstream task to use it for predictions and inference.
/var/folders/z7/j9rhznl118zbt_41ygz3rgyc0000gn/T/ipykernel_90396/853941217.py:30: DeprecationWarning: `SetFitTrainer` has been deprecated and will be removed in v2.0.0 of SetFit. Please use `Trainer` instead.
  trainer = SetFitTrainer(
/opt/anaconda3/lib/python3.12/site-packages/executing/executing.py:713: DeprecationWarning: ast.Str is deprecated and will be removed in Python 3.14; use ast.Constant instead
  rig

OSError: [Errno 30] Read-only file system: 'tmp_trainer'